## Import modules

In [1]:
import hyperspy.api as hp
import SpectrumImage
import SpectrumImagePlotter
import Spectrum
import os
import Image
import ImagePlotter
import ImageDisplay
import matplotlib.pyplot as plt
%matplotlib qt

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Import spectrum image and point spread function

In [2]:
# Define folder and filename for spectrum image
folder = '/home/user/datafolder'
filebase = 'EELS Spectrum Image.dm3'

# Define folder to save the processed data in
savefolder = os.path.join(folder, 'Processed')

# Define folder and filename of the point spread function
PSFfolder = folder
PSFfilename = 'Spectrum_ZLP.csv'

In [3]:
# Load spectrum image from dm3 using hyperspy
s = hp.load(os.path.join(folder, filebase))
dispersion=s.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension.TagGroup2.Scale
eels = SpectrumImage.EELSSpectrumImage(s.data, dispersion=dispersion)

# Load point spread function spectrum as an EELSSpectrum object from csv
PSF = Spectrum.EELSSpectrum.LoadFromCSV(os.path.join(PSFfolder, PSFfilename))

# Print the FWHM of the point spread function
print PSF.FindFW(0.5)

0.0534713001649


## Threshold and deconvolve SI

In [4]:
# Threshold the spectrum image based on the zero loss peak to eliminate data with low signal and show the thresholded 
# image (threshold applies a mask) (optional)
eels.Threshold(10000)
plt.imshow(eels.data[:,:,eels.ZLP])
plt.show()

In [5]:
# Deconvolve data using Richardson-Lucy algorithm, define the number of iterations to use (eg, 5), the point spread 
# function and the number of multiprocessing threads to use (eg, 8)
RLiterations = 5
eels2 = eels.RLDeconvolution(RLiterations, PSF, threads=8)

Beginning deconvolution...
Done 2 iterations!


## Plot SI data

In [6]:
# Plot the original spectrum image (p1) and the deconvolved spectrum image (p2) (interactive)
p1 = SpectrumImagePlotter.SpectrumImagePlotter(eels, filepath=os.path.join(savefolder))
p2 = SpectrumImagePlotter.SpectrumImagePlotter(eels2, filepath=os.path.join(savefolder, str(RLiterations) + 'RL'))
p1.ShowPlot()

SpectrumPlotter.py:83: RuntimeWarning: divide by zero encountered in double_scalars
  eV = h*c/(eC*nm)*1e9


In [ ]:
#Plot spectrum image using hyperspy interface
def hspyplotSI(SIx, SI1metadata, ZLP, dispersion):
	SI = hp.signals.Spectrum(SIx)
	SI.metadata = SI1metadata
	SI.axes_manager[0].name = 'x'
	SI.axes_manager[0].units = 'nm'
	SI.axes_manager[1].name = 'y'
	SI.axes_manager[1].units = 'nm'
	SI.axes_manager[2].name = 'Energy loss'
	SI.axes_manager[2].units = 'eV'
	SI.axes_manager[2].scale = dispersion
	SI.axes_manager[2].offset = -ZLP * dispersion
	SI.plot()
	return SI

s_deconv = hspyplotSI(eels2.data.data, s.metadata, eels2.ZLP, dispersion)

# Save as RPL format using hyperspy
f1 = os.path.join(folder, s.metadata.General.original_filename)
f2 = f1.encode('ascii','replace')[:-4]+'_%d' %RLiterations + 'RL.rpl'
s_deconv.save(f2)